In [1]:
import keras  
from gensim.models import Word2Vec

Using TensorFlow backend.
D:\ANACONDA\envs\tensorflownt\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np   
from keras.utils import plot_model,np_utils  
from keras.preprocessing.image import ImageDataGenerator  
from keras.models import *  
from keras.layers import *  
from keras import backend as K  
import h5py 
import random

# Load Data

In [3]:
filename = "cp.txt"  
raw_text = open(filename).read()  
raw_text = raw_text.lower()

## Chars to interger vectors: build rnn's input

In [4]:
chars = sorted(list(set(raw_text)))  
char_to_int = dict((c, i) for i, c in enumerate(chars))  
int_to_char = dict((i, c) for i, c in enumerate(chars)) 

In [5]:
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '…']


In [6]:
n_chars = len(raw_text)  
n_vocab = len(chars)

In [7]:
print('vocal: ',n_vocab)

vocal:  54


## cut the text in semi-redundant sequences of sqe_length characters

In [8]:
seq_length = 36  #how to decide
step = 3
dataX = []  
dataY = [] 

In [9]:
for i in range(0, n_chars - seq_length, step):  
    seq_in = raw_text[i:i + seq_length]  
    seq_out = raw_text[i+1:i+1+seq_length] 
    dataX.append([char_to_int[char] for char in seq_in])  
    #dataY.append(char_to_int[seq_out])
    dataY.append([char_to_int[c] for c in seq_out])

In [10]:
print(dataX[:3])
print(dataY[:3])

[[43, 44, 33, 36, 36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1], [36, 36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29], [37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29, 36, 1, 37]]
[[44, 33, 36, 36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30], [36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29, 36], [49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29, 36, 1, 37, 49]]


In [11]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns) 

Total Patterns:  34404


## reshape X to be [samples, time steps, features] , and so do Y

In [12]:
X = np.zeros((n_patterns, seq_length, n_vocab), dtype = np.bool)
Y = np.zeros((n_patterns, seq_length, n_vocab), dtype=np.bool)
for i, sentence in enumerate(dataX):
    for t, char in enumerate(sentence):
        X[i, t, char] = 1
for i, sentence in enumerate(dataY):
    for t, char in enumerate(sentence):
        Y[i, t, char] = 1    

# Build rnn Model

In [13]:
K.clear_session()

In [14]:
model = Sequential()  

model.add(LSTM(128, input_dim=n_vocab,return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, input_shape=(None, 54), return_sequences=True)`
  This is separate from the ipykernel package so we can avoid doing imports until


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 128)         93696     
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 54)          6966      
_________________________________________________________________
activation_1 (Activation)    (None, None, 54)          0         
Total params: 232,246
Trainable params: 232,246
Non-tra

In [15]:
#import pydot
#import graphviz
#plot_model(model, to_file='model.png') 

# Training the Model

In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

nit = 5   # number of training iterations
nepoch_per_it = 20  # number of epochs per iterations
# Main iteration loop
for it in range(nit):
    # Continue the fit of the model
    init_epoch = it*nepoch_per_it
    model.fit(X, Y, epochs=nepoch_per_it, batch_size=128,verbose=0)

    #Output generated text after each iteration
    print('-'*50)
    print("iter = ",it)
    start = random.randint(0, n_chars - seq_length -1)
    generated = ''
    sentence = raw_text[start:start+seq_length]
    generated +=sentence
    print('----- Generating with seed: "' + sentence + '"')
    for i in range(400):
        x_pred = np.zeros((1, seq_length, n_vocab))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_int[char]]=1
        preds = model.predict(x_pred, verbose=0)[0]
        next_int = sample(preds[len(sentence)-1])
        next_char = int_to_char[next_int]
        generated += next_char
        sentence = sentence[1:]+next_char
    print(generated)
           
model.save('lyrics_pre.h5')  

--------------------------------------------------
iter =  0
----- Generating with seed: "rong to 
ask, who does this belong t"


D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


rong to 
ask, who does this belong to a place what it is you're talking that outer space and sing to me
the spiel waiting for you
and if you love me, won't you let me through
let me go
but then i ever want to fall
could have been for i love it when you come over to the heaving of the saint
said if i can a believer
don’t say it was all yellow

you can be any heavy out of the things and one on you
oh


come stired in my heart i couldn
--------------------------------------------------
iter =  1
----- Generating with seed: "t's only your imagination
it's only "
t's only your imagination
it's only your imagination
it's only all of the throked it break your heart


poolly half if something that my gooh gone for

and you say, "oh, say
how come people struggle how come people suffer eyes
in the new sun in
the more you windo
so low 
didn't believe i'm in
i've got to tell you in my loudest tones
i'll caurblieldip on the radced
be dark felon the thousandse and shorning to do
so i wanna live in a

# Generate Lyrics

In [18]:
generated = ''
#The seed
sentence = 'tuesday'
generated +=sentence
print('----- Generating with seed: "' + sentence + '"')
print ('The generated text is:\n')
for i in range(400): ##How long
    x_pred = np.zeros((1, len(sentence), n_vocab))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_to_int[char]]=1
    preds = model.predict(x_pred, verbose=0)[0]
    next_int = sample(preds[len(sentence)-1])
    next_char = int_to_char[next_int]
    generated += next_char
    sentence = sentence[1:]+next_char
print(generated)

----- Generating with seed: "tuesday"
The generated text is:



D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


tuesday
but in vooles
i need a painter stays it's true
i'm so run and sills before the holes now
it could see it all 'bre the world
and they shine way
but nothing under the south when you'll never start with the world in mine?
and i can’t get down me
and in my world
all that you shiver that you’re going to be moving at all


lovers find me
so i turn the new all i want to roll a lifetisnomy of eyes

but g
